In [ ]:
##Title Imports
from random import randint, sample, random, uniform
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

In [ ]:
class Peixe():
  
  def __init__(self, position, peso):
    self.position = position
    self.peso = peso
    self.deltaF = 0
    self.fitness = 0
    self.deltaPosicao = []
    self.melhorouPeso = False
    
  def updatePosition(self, position):
    self.position = position
  
  def updateMelhorouPeso(self, boolean):
    self.melhorouPeso = boolean
  
  def updateDeltaF(self, deltaF):
    self.deltaF = deltaF
  
  def updatePeso(self, peso):
    self.peso = peso
  
  def updateFitness(self, fitness):
    self.fitness = fitness
  
  def updateDeltaPosicao(self, deltaPosicao):
    self.deltaPosicao = deltaPosicao

In [ ]:
def criadorDePeixes(numPeixes, dimensoes, peso, feasibleBounds1,feasibleBounds2):
  peixes = []
  for i in range(0,numPeixes):
    position = []
    peso = peso
    for j in range(0, dimensoes):
      position.append(uniform(feasibleBounds1,feasibleBounds2))
    
    p = Peixe(position, peso)
    peixes.append(p)
  
  return peixes

In [ ]:
##Title Functions CEC 2013

def sphere(vetor):
  resultado = 0
  
  for i in vetor:
    resultado += i**2
  
  return resultado

def rastrigin(vetor):
  resultado = 0
  
  for i in vetor:
    numero = 2*3.1415*i
    p = (numero/180)*math.pi
    resultado+= (i**2) - (10 * math.cos(p)) + 10
    
  return resultado

def rosenbrock(vetor):
  resultado = 0
  
  for i in range(0,(len(vetor)-1)):
    resultado += 100*(vetor[i+1] - vetor[i]**2)**2 + (vetor[i] - 1)**2
    
  return resultado


def FFitness(vetor, tipo):
  if tipo == 1:
    return sphere(vetor)
  elif tipo == 2:
    return rastrigin(vetor)
  else:
    return rosenbrock(vetor)
  
  

## FUNÇÃO MEDIA DE VETORES

def mediaVetorGBest(vetorDeVetores):
  
  quantVetores = len(vetorDeVetores)
  tamanhoVetor = len(vetorDeVetores[0])
  vetorResultado = []
  for i in range(0,tamanhoVetor):
    soma = 0
    for j in range(0, quantVetores):
      soma += vetorDeVetores[j][i]
    soma/=quantVetores
    vetorResultado.append(soma)
    
  return vetorResultado

In [ ]:
def PositionNeighborhood(vetor, dimension, sind, feasibleBounds1, feasibleBounds2):
  
  new_pos = [0.0]*dimension
  for i in range(dimension):
    new_pos[i] = vetor[i] + (sind * uniform(-1,1))
    if new_pos[i] < feasibleBounds1:
      new_pos[i] = feasibleBounds1
    elif new_pos[i] > feasibleBounds2:
      new_pos[i] = feasibleBounds2
      
  return new_pos



def movimentoIndividual(peixes, dimensoes, sind, feasibleBounds1, feasibleBounds2, tipoFuncao, pesoMin,
                       GBest, valueGBest, avaliacoes):
  maiorDeltaF = -15
  
  
  for i in peixes:
      
    n = PositionNeighborhood(i.position, dimensoes, sind, feasibleBounds1, feasibleBounds2)

    fitnessN = FFitness(n, tipoFuncao)
    avaliacoes += 1
    fitnessX = FFitness(i.position, tipoFuncao)
    avaliacoes += 1

    if fitnessN < fitnessX:
      deltaF = abs(fitnessN - fitnessX)
      i.updateDeltaF(deltaF)
      if deltaF > maiorDeltaF:
        maiorDeltaF = deltaF
      i.updateFitness(fitnessN)
      delta_pos = [0.0]*dimensoes
      for idx in range(dimensoes):
        delta_pos[idx] = n[idx] - i.position[idx]
      i.updateDeltaPosicao(delta_pos)
      i.updatePosition(n)
      
      if fitnessN < valueGBest:
        valueGBest = fitnessN
        GBest = n
      
    else:
      i.updateDeltaPosicao([0.0]*dimensoes)
      i.updateDeltaF(0)
      
      if fitnessX < valueGBest:
        valueGBest = fitnessX
        GBest = i.position
    

  for i in peixes:
    novoPeso = i.peso + (i.deltaF/maiorDeltaF)
    if novoPeso > i.peso:
      i.updateMelhorouPeso(True)
    else:
      i.updateMelhorouPeso(False)
    if novoPeso < pesoMin:
      novoPeso = pesoMin
    i.updatePeso(novoPeso)

  return peixes, GBest, valueGBest, avaliacoes
  

def movimentoInstintivo(peixes, dimensoes, feasibleBounds1, feasibleBounds2):
  
  m = [0.0]*dimensoes
  somatorioDeltaF = 0
  
  for i in peixes:
    somatorioDeltaF = i.deltaF
    for idx in range(dimensoes):
      m[idx] += (i.deltaPosicao[idx] * i.deltaF)
      
  for idx in range(dimensoes):
    if somatorioDeltaF != 0:
      m[idx] = m[idx]/somatorioDeltaF
      
  for i in peixes:
    novaPosicao = [0.0]*dimensoes
    for idx in range(dimensoes):
      novaPosicao[idx] = i.position[idx] + m[idx]
      if novaPosicao[idx] < feasibleBounds1:
        novaPosicao[idx] = feasibleBounds1
      elif novaPosicao[idx] > feasibleBounds2:
        novaPosicao[idx] = feasibleBounds2
    
    i.updatePosition(novaPosicao)
  return peixes  
    
def movimentoVolutivo(peixes, dimensoes, feasibleBounds1, feasibleBounds2, svol):
  
  ## Calcular Barycenter
  b = [0.0]*dimensoes
  somatorioPeso = 0
#   f = -1
  for i in peixes:
#     f+=1
    somatorioPeso += i.peso
    for idx in range(dimensoes):
      b[idx] += (i.position[idx] * i.peso)
#       if f == 0 and idx == 0:
#         print("POSICAO: ", i.position[idx])
#         print("PESO: ", i.peso)
#         print("Bary: ", b[idx])
  
  for idx in range(dimensoes):
#     if idx == 0:
#       print("Bary: ", b[idx])
#       print("SomatorioPeso: ", somatorioPeso)
    b[idx] = b[idx] / somatorioPeso
#     if idx == 0:
#       print("BaryFim: ", b[idx])
  
#   f = -1
  for i in peixes:
#     f+=1
    novaPosicao = [0.0]*dimensoes
    for idx in range(dimensoes):
      if i.melhorouPeso:
        novaPosicao[idx] = i.position[idx] - ((i.position[idx] - b[idx]) * svol * uniform(0,1))
#         if f == 0 and idx == 0:
#           print("1Posicao: ", i.position[idx])
#           print("Barycenter: ", b[idx])
#           print("SomatorioPeso: ", somatorioPeso)
#           print("Unifor: ", uniform(0,1))
#           print("NovaPosicao: ", novaPosicao[idx])
      else:
        
        novaPosicao[idx] = i.position[idx] + ((i.position[idx] - b[idx]) * svol * uniform(0,1))
#         if f == 0 and idx == 0:
#           print("2Posicao: ", i.position[idx])
#           print("Barycenter: ", b[idx])
#           print("SomatorioPeso: ", somatorioPeso)
#           print("Unifor: ", uniform(0,1))
#           print("NovaPosicao: ", novaPosicao[idx])
      if novaPosicao[idx] < feasibleBounds1:
        novaPosicao[idx] = feasibleBounds1
      elif novaPosicao[idx] > feasibleBounds2:
        novaPosicao[idx] = feasibleBounds2
        
    
    i.updatePosition(novaPosicao)
  
  return peixes



In [ ]:
def FSS(nPeixes, dimensoes, peso, feasibleBounds1,feasibleBounds2, avaliacoes, tipoFuncao, sindMin, sindMax, 
        svolMin, svolMax, pesoMin):
    
  l = 0
  GBest = []
  valueGBest = 999999999999
  vetorGBest = []
  sind = sindMax
  svol = svolMax
    
  peixes = criadorDePeixes(nPeixes, dimensoes, peso, feasibleBounds1, feasibleBounds2)
  
  iteracoes = (avaliacoes/nPeixes)/2
  
  tempGBest = GBest
  tempValueGBest = valueGBest
  
  while l < avaliacoes:
#     print(peixes[0].position)
    peixes, tempGBest, tempValueGBest, l = movimentoIndividual(peixes, dimensoes, sind, feasibleBounds1, feasibleBounds2, 
                                                       tipoFuncao, pesoMin, tempGBest, tempValueGBest, avaliacoes=l) 
#     print(peixes[0].position)
    peixes = movimentoInstintivo(peixes, dimensoes, feasibleBounds1, feasibleBounds2)
#     print(peixes[0].position)
    peixes = movimentoVolutivo(peixes, dimensoes, feasibleBounds1, feasibleBounds2, svol)
#     print(peixes[0].position) 
    sind = sind - ((sindMax - sindMin)/iteracoes)
    svol = svol - ((svolMax - svolMin)/iteracoes)
    
    if tempValueGBest < valueGBest:
#       print(tempValueGBest)
      valueGBest = tempValueGBest
      GBest = tempGBest
    vetorGBest.append(valueGBest)
  
#   print(valueGBest)
  return valueGBest, GBest, vetorGBest
    

In [ ]:
##Title Parametros
# 30 peixes
# 30 dimensoes
# 500000 avaliacoes
# step individual decaindo de 0.1 a 0.001
# step voluntivo decaindo de 0.01 a 0.001
results = pd.DataFrame()
resultsValue = pd.DataFrame()

nPeixes = 30
dimensoes = 30
peso = 2
sindMin = 0.001
sindMax = 0.1
svolMin = 0.001
svolMax = 0.01
avaliacoes = 500000
pesoMin = 1
##Tipo função - 1 - sphere / 2 - rastrigin / 3 - rosenbrock



for k in range(1,4):
  tipoFuncao = k
  if k == 1:
    feasibleBounds1 = -100
    feasibleBounds2 = 100
  elif k == 2:
    feasibleBounds1 = -30
    feasibleBounds2 = 30
  else:
    feasibleBounds1 = -5.12
    feasibleBounds2 = 5.12
  vetorTempVetorGBest = []
  vetorTempValueGBest = []
  for j in tqdm(range(0,30)):

    valueGBest, GBest, vetorGBest = FSS(nPeixes, dimensoes, peso, feasibleBounds1,feasibleBounds2, avaliacoes, tipoFuncao, sindMin, sindMax, svolMin, svolMax, pesoMin)
    vetorTempVetorGBest.append(vetorGBest)
    vetorTempValueGBest.append(valueGBest)
  #vetorVetorGBest.append(mediaVetorGBest(vetorTempVetorGBest))
  results[f'sim_{k}'] = mediaVetorGBest(vetorTempVetorGBest)
  resultsValue[f'sim_{k}'] = vetorTempValueGBest
  #vetorValueGBest.append(vetorTempValueGBest)
    

results.to_csv(f'fss_.csv', index=False)
resultsValue.to_csv(f'fss_2.csv', index=False)

100%|██████████| 30/30 [09:41<00:00, 19.19s/it]
